In [1]:
import pandas as pd
import numpy as np
import requests
import io
import datetime as dt
today = dt.datetime.today().strftime('%Y-%m-%d')
import openpyxl
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

In [8]:
client = ['d_evertson',21982304

         ]
client_name = client[0]
addepar_id = client[1]

# Access and organize Addepar data
addepar_view ='396474' #CSP Main View
start_date = today
end_date = today
AUTH_KEY = '9087f51f-e3e1-4d53-87f1-ab90b67b1d55'
AUTH_SECRET = 'vQ18ffmE9fBRcnbvZlFdBUXkRH89YcDl0kIlnNek'
r = requests.get('https://lido.addepar.com/api/v1/portfolio/views/{}/results?portfolio_id={}&portfolio_type=entity&output_type=csv&start_date={}&end_date={}&addepar_firm=222'.format(
                addepar_view, 
                addepar_id, 
                start_date, 
                end_date),
                 auth=('{}'.format(AUTH_KEY),
                       '{}'.format(AUTH_SECRET)
                    )).content  # request to addepar
# Read the file, store as DataFRame
x = pd.read_csv(io.StringIO(r.decode('utf-8')))
#Organize DataFrame
close_prices = x[['Symbol','Expiration Date','Strike Price (USD)','Security','Quantity','Original Cost Basis (USD)','Value (USD)']].copy()
close_prices['Price'] = close_prices['Value (USD)'].div(close_prices['Quantity'])
# Write data to existing options summary workbook
wb = Workbook()
ws = wb.active
xl_file=r"C:\Users\mreis\OneDrive - Lido Advisors, LLC\Concentrated Stock\Lido CSP\CURRENT PORTFOLIOS\options summaries\{}_options_summary.xlsx".format(client_name)
wb=openpyxl.load_workbook(xl_file)
ws=wb.worksheets[1]
for row in np.arange(0,len(close_prices),1):
    for col in np.arange(0,len(close_prices.columns),1):
        ws.cell(row=row+2,column=col+5).value=close_prices.iloc[row,col]
wb.save(xl_file)
wb.close()